Simulator: Sky Background
=========================

This script simulates `Imaging` of a galaxy where the sky background is not subtracted from the image and therefore
appears in the dataset.

It is used to demonstrate sky background modeling in 
the `autogalaxy_workspace/*/imaging/modeling/features/sky_background.py` example.

__Model__

The galaxy uses light profiles where:

 - The galaxy's bulge is an `Sersic`.
 - The galaxy's disk is an `Exponential`.

__Start Here Notebook__

If any code in this script is unclear, refer to the `simulators/start_here.ipynb` notebook.

In [ ]:
%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

from os import path
import autogalaxy as ag
import autogalaxy.plot as aplt

__Dataset Paths__

The path where the dataset will be output.

In [ ]:
dataset_type = "imaging"
dataset_name = "sky_background"

dataset_path = path.join("dataset", dataset_type, dataset_name)

__Simulate__

Simulate the image using the `Grid2DIterate` object, which is a grid of (y,x) coordinates that is iteratively
where the sub-size of the grid is increased until the input fractional accuracy of 99.99% is met.

In [ ]:
grid = ag.Grid2DIterate.uniform(
    shape_native=(100, 100),
    pixel_scales=0.1,
    fractional_accuracy=0.9999,
    sub_steps=[2, 4, 8, 16, 24],
)

Simulate a simple Gaussian PSF for the image.

In [ ]:
psf = ag.Kernel2D.from_gaussian(
    shape_native=(11, 11), sigma=0.1, pixel_scales=grid.pixel_scales
)

Create the simulator for the imaging data, which defines the exposure time, background sky, noise levels and psf.

The input `subtract_background_sky=False` ensures the background sky is not subtracted from the image, but is 
included in the data and used to estimate its noise-map.

The `background_sky_level` is also increased to 5.0 electrons per second, which is much higer than the value of 
0.1 electrons per second used in previous examples. This is to better illustrate the sky background in the image
and how it can be modeled.

In [ ]:
simulator = ag.SimulatorImaging(
    exposure_time=300.0,
    psf=psf,
    background_sky_level=5.0,
    subtract_background_sky=False,
    add_poisson_noise=True,
)

__Galaxies__

Setup the galaxy with a bulge (elliptical Sersic) for this simulation.

In [ ]:
galaxy = ag.Galaxy(
    redshift=0.5,
    bulge=ag.lp.Sersic(
        centre=(0.0, 0.0),
        ell_comps=ag.convert.ell_comps_from(axis_ratio=0.9, angle=45.0),
        intensity=1.0,
        effective_radius=0.8,
        sersic_index=4.0,
    ),
)

Use these galaxies to generate the image for the simulated `Imaging` dataset.

In [ ]:
galaxies = ag.Galaxies(galaxies=[galaxy])
galaxies_plotter = aplt.GalaxiesPlotter(galaxies=galaxies, grid=grid)
galaxies_plotter.figures_2d(image=True)

Pass the simulator galaxies, which creates the image which is simulated as an imaging dataset.

In [ ]:
dataset = simulator.via_galaxies_from(galaxies=galaxies, grid=grid)

Plot the simulated `Imaging` dataset before outputting it to fits.

In [ ]:
dataset_plotter = aplt.ImagingPlotter(dataset=dataset)
dataset_plotter.subplot_dataset()

__Output__

Output the simulated dataset to the dataset path as .fits files.

In [ ]:
dataset.output_to_fits(
    data_path=path.join(dataset_path, "data.fits"),
    psf_path=path.join(dataset_path, "psf.fits"),
    noise_map_path=path.join(dataset_path, "noise_map.fits"),
    overwrite=True,
)

__Visualize__

Output a subplot of the simulated dataset, the image and the galaxies quantities to the dataset path as .png files.

In [ ]:
mat_plot = aplt.MatPlot2D(output=aplt.Output(path=dataset_path, format="png"))

dataset_plotter = aplt.ImagingPlotter(dataset=dataset, mat_plot_2d=mat_plot)
dataset_plotter.subplot_dataset()
dataset_plotter.figures_2d(data=True)

galaxies_plotter = aplt.GalaxiesPlotter(
    galaxies=galaxies, grid=grid, mat_plot_2d=mat_plot
)
galaxies_plotter.subplot()

__Plane Output__

Save the `Plane` in the dataset folder as a .json file, ensuring the true light profiles and galaxies
are safely stored and available to check how the dataset was simulated in the future. 

This can be loaded via the method `galaxies = ag.from_json()`.

In [ ]:
ag.output_to_json(
    obj=galaxies,
    file_path=path.join(dataset_path, "galaxies.json"),
)

The dataset can be viewed in the folder `autogalaxy_workspace/imaging/simple__sersic`.